# Polling stations scraper

This script was written to extract polling stations information from psleci.nic.in website for non-commercial 
analyses purposes.

Execute `pip install -r requirements-polling-stations-scraper.txt` before continuing.

In [1]:
import pandas as pd # pandas, because easy data manipulation
import requests as req # to make requests with
from multiprocessing import Pool, Manager, Process


We are only looking for poll-stations in Bangalore, so we will `S` variable  will remain intact in the following URL.
http://psleci.nic.in/pslamf.aspx?S=S10&A=150&P=1

We will increment `A` and `P` variables, till we do not get new data.
The website doesn't respond with a HTTP 4xx code when given outlandish variables.
For example, when we look for the Ten thousandth polling station in Belgaum http://psleci.nic.in/pslamf.aspx?S=S10&A=150&P=1, 
it doesn't give a 4xx status code.

In [2]:
completed_url_list = []

In [3]:
data = []

In [ ]:
URL = 'http://psleci.nic.in/pslamf.aspx?S=S10&A=150&P={:d}'
url_list = []


def scrape_polling_station(url):
    try:
        
        if url not in completed_url_list:
            temp_ps = pd.read_html(url)
            ps_df = temp_ps[0].copy()
            ps_df.loc[1:4, :]= ps_df.loc[1:4, 0].apply(lambda v: pd.Series(v.split(':')))
            ps_df = ps_df.drop(0, axis=0).T
            ps_df.columns = ps_df.loc[0]
            ps_df = ps_df.drop(0, axis=0)
            
            completed_url_list.append(url)
            data.append(ps_df)
            return ps_df
        else:
            print("cached ps: ", ps_name)
    except Exception as e:
        print(e)
        print(url)
        print(len(data))
        scrape_polling_station(url)

        
def append_data(datum):
    '''
        appends data to multiprocessing collector object
    '''
    data.append(datum)

# generating url list for multiprocessing
# for ind_district in range(1, 100):
for ind_polling_station in range(1, 5000):
    url_list.append(URL.format(ind_polling_station))
print('generated url list')

pool=Pool()
for url in url_list:
    r = pool.apply_async(scrape_polling_station, 
                  (url,),
                  callback=append_data).get()

generated url list


In [8]:
pd.concat(data).dropna(how='all').to_csv("polling-stations-bangalore.csv", index=False)

(402, 33)